In [96]:
import re
import numpy as np
import codecs
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import glob, os


class CurrentClasses:
    def __init__(self, class_num, word_count, item_count, class_values):
        self._weights = np.random.rand(word_count, class_num)
        self._b = np.random.rand(len(class_values))
        self._class_num = class_num
        self._word_count = word_count
        self._item_count = item_count
        self._alfa = 0.01
        self._lamd = 0.01
        self._class_values = class_values;
        
        self._y = (np.arange(np.max(class_values) + 1) == class_values[:, None]).astype(float)
        
       
    
    def currentFunction(self, elements):
        return (elements.dot(self._weights))
        #return (elements.dot(self._weights) + self._b)
    
    def softMax(self, z):
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T
    
    def toClasslabel(self, z):
        return z.argmax(axis=1)
    
    def crossEntropy(self, output):
        return - np.sum(np.log(output) * (self._y), axis=1)
    
    def cost(self, output):
        return np.mean(self.crossEntropy(output))
    
    def recalcWeights(self, y_values, softmax, x_values):
        w_add = np.zeros((self._word_count, self._class_num))
        for j in range(0, self._class_num):
            for i in range(0, self._item_count):
                if ( self._class_values[i] == y_values[j] ):
                    w_add[i][j] += 1;
                w_add[i][j] -= x_values[i][j]*softmax[i][j];
        for j in range(0, self._class_num):
            for i in range(0, self._item_count):
                self._weights[i][j] -= self._alfa*((-1)*w_add[i][j]/self._item_count - self._lamd*self._weights[i][j]);
    
 

class ReviewItem:
    def __init__(self, pos, filename):
        self._pos = pos;
        self._allwords = [];
        m = re.search('([\d]+?)_([\d]+?)\.txt$', filename)
        self._id = m.group(1)
        self._rank = m.group(2)
        f = codecs.open(filename, "r")
        for line in f:
            line_v = line.strip().lower().split(",")
            line = ' '.join(line_v[1:])
            words = re.split('[^a-z\d]',line)
            if '' in words:
                words.remove('')
            for word in words:
                self._allwords.append(word)
        self._wcount = dict()
        self._words = []
        
    def recalcWIndex(self, windex, wcount):
        self._words = dict()
        for word in self._allwords:
            if windex[word] not in self._words:
                self._words[windex[word]] = 0;
            self._words[windex[word]] += 1;
    
        #for word_index in self._words:
            #self._words[word_index] /= wcount[word_index]
            #self._words[word_index] /= len(self._allwords)
        self._allwords.clear()
       
        
       





In [82]:
pos_file_count = 0;
reviews = []

limit_read = 100;
for file in os.listdir("aclImdb/train/pos") :
    if file.endswith(".txt") and len(reviews) < limit_read :
        pos_file_count += 1;
        reviews.append(ReviewItem(1, "aclImdb/train/pos/" + file))

for file in os.listdir("aclImdb/train/neg") :
    if file.endswith(".txt") and len(reviews) < 2*limit_read:
        pos_file_count += 1;
        reviews.append(ReviewItem(0, "aclImdb/train/neg/" + file))

test_reviews = []

for file in os.listdir("aclImdb/test/pos") :
    if file.endswith(".txt") and len(test_reviews) < limit_read :
        pos_file_count += 1;
        test_reviews.append(ReviewItem(1, "aclImdb/test/pos/" + file))

for file in os.listdir("aclImdb/test/neg") :
    if file.endswith(".txt") and len(test_reviews) < 2*limit_read :
        pos_file_count += 1;
        test_reviews.append(ReviewItem(0, "aclImdb/test/neg/" + file))
        

        
windex = dict()
wcount = dict()
word_index = 0;
review_pos = 0;
class_values = np.zeros(len(reviews))

for review in reviews:
    class_values[review_pos] = review._pos;
    review_pos += 1
    for word in review._allwords:
        
        if word not in windex:
            windex[word] = word_index;
            wcount[word_index] = 0
            word_index += 1;
        wcount[windex[word]] += 1
        
x_values = np.zeros((len(reviews), len(windex)))
review_pos = 0;
for review in reviews:
    for word in review._allwords:
        x_values[review_pos][windex[word]] = .1;
        
    review_pos += 1
            
    #review.recalcWIndex(windex, wcount)
            
print(pos_file_count);        

print(len(windex))

400
6235


In [98]:
x = CurrentClasses(2, len(windex), len(reviews), class_values)

diff = 9999999;
prev_diff = 100000000
c_val = 0
same_iter = 0
debug_review_id = -1

for _ in range(100) :
    print("diff and prev:", diff, prev_diff) 
    if prev_diff == diff:
        same_iter += 1;
    else:
        same_iter = 0;
    prev_diff = diff
    c_val_array = [0]*len(windex)
   
    funcValue = x.currentFunction(x_values)
    smax = x.softMax(funcValue)
    # print('softmax:', smax)
    print('predicted class labels: ', x.toClasslabel(smax))
    y_values = x.toClasslabel(smax);
    xent = x.crossEntropy(smax)
    #print('Cross Entropy:', xent)
    diff = c_val;
    
    diff = x.cost(smax)
    #print('Cost: ', J_cost)
    
    x.recalcWeights(y_values, smax, x_values)
    
    print("last_diff:", diff)

diff and prev: 9999999 100000000
predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706534347148
diff and prev: 0.706534347148 9999999
predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706531571468
diff and prev

predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706437469284
diff and prev: 0.706437469284 0.706440229444
predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706434709584
diff and prev: 0.706434709584 0.7064374

predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.70636311952
diff and prev: 0.70636311952 0.706365867188
predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706360372317
diff and prev: 0.706360372317 0.706363119

predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706267245857
diff and prev: 0.706267245857 0.706269977111
predicted class labels:  [0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0
 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0
 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1]
last_diff: 0.706264515076
diff and prev: 0.706264515076 0.7062672